# Capstone Project - The Battle of the Neighborhoods (Week 2)

### Opening a new Cafe in Mumbai, India

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Coffee Shop** in **Mumbai**, India.

**Question**- What should be an appropriate location to open a new cafe in Mumbai city?

Now, in order for a cafe to be profitable, there should be enough customers, and in order to have enough customers, it is not worth setting up customers is vicinity of existing ones. Since there are lots of restaurants already  we will try to detect **locations that are not already crowded with restaurants**. 

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria.

## Data <a name="data"></a>

1. Mumbai neighborhood dataset from wikipedia. This data contains the latitude and longitude coordinates for all neighborhoods https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai
2. Foursqaure API- To obtain venue data of neighborhoods.

### 1.Import all the needed libraries.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

### 2. Scrap data from Wikipedia page into a DataFrame

In [3]:
url='https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai'
read_table= pd.read_html(url,header=[0])
df=read_table[0]
df.head(10)

,Area,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400
1,"Chakala\tAndheri,",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270
5,Marol,"Andheri,Western Suburbs",19.119219,72.882743
6,Sahar,"Andheri,Western Suburbs",19.098889,72.867222
7,Seven Bungalows,"Andheri,Western Suburbs",19.129052,72.817018
8,Versova,"Andheri,Western Suburbs",19.120000,72.820000
9,Mira Road,"Mira-Bhayandar,Western Suburbs",19.284167,72.871111


In [37]:
# Check the shape of the dataframe
df.shape

(93, 4)

In [5]:
# save the DataFrame as CSV file
df.to_csv("mumbai_df.csv", index=False)

## Methodology <a name="methodology"></a>

1. Scraping the wikipedia web page to obtain neighborhood list
2. Obtain the above data and store it in pandas dataframe along with latitude and longitude values.
3. Use Foursquare API to get venue data.
4. Group data by neighborhood and take mean of the frequency of occurance of each category.
5. Filter venue by Coffee Shops
6. Perform clustering on the data by using k-means clustering algorithm.
7. Visualize the clusters in a map using Folium.

###  3. Create a map of Mumbai with neighborhoods superimposed on top

In [6]:
#Get the location coordinates of Mumbai city
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="mum_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai City are 18.9387711, 72.8353355.


In [39]:
# create map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, area, location in zip(df['Latitude'], df['Longitude'], df['Area'], df['Location']):
    label = '{}, {}'.format(area, location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

In [8]:
# save the map as HTML file
map_mumbai.save('map_mumbai.html')

### 4. Use the Foursquare API to explore the neighborhoods

In [9]:
# define Foursquare Credentials and Version
CLIENT_ID = 'XUTTAUHJ5CMQPNOKGTWA2GEMYRHPI41CHI2PKM3QN2NNGXR2' # your Foursquare ID
CLIENT_SECRET = 'TJUBFZFEHYQD3L5U1IURPV4PQAXT0UYJQYJ2BQDLXPHQ3LFS' # your Foursquare Secret
VERSION = '20200610' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XUTTAUHJ5CMQPNOKGTWA2GEMYRHPI41CHI2PKM3QN2NNGXR2
CLIENT_SECRET:TJUBFZFEHYQD3L5U1IURPV4PQAXT0UYJQYJ2BQDLXPHQ3LFS


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [10]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

mumbai_venues = getNearbyVenues(names=df['Area'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Amboli
Chakala	Andheri,
D.N. Nagar
Four Bungalows
Lokhandwala
Marol
Sahar
Seven Bungalows
Versova
Mira Road
Bhayandar
Uttan
Bandstand Promenade
Kherwadi
Pali Hill
I.C. Colony
Gorai
Dahisa
Aarey Milk Colony
Bangur Nagar
Jogeshwari West
Juhu
Charkop
Poisar
Mahavir Nagar
Thakur village
Pali Naka
Khar Danda
Dindoshi
Sunder Nagar
Kalina
Naigaon
Nalasopara
Virar
Irla
Vile Parle
Bhandup
Amrut Nagar
Asalfa
Pant Nagar
Kanjurmarg
Nehru Nagar
Nahur
Chandivali
Hiranandani Gardens
Indian Institute of Technology Bombay campus
Vidyavihar
Vikhroli
Chembur
Deonar
Mankhurd
Mahul
Agripada
Altamount Road
Bhuleshwar
Breach Candy
Carmichael Road
Cavel
Churchgate
Cotton Green
Cuffe Parade
Cumbala Hill
Currey Road
Dhobitalao
Dongri
Kala Ghoda
Kemps Corner
Lower Parel
Mahalaxmi
Mahim
Malabar Hill
Marine Drive
Marine Lines
Mumbai Central
Nariman Point
Prabhadevi
Sion
Walkeshwar
Worli
C.G.S. colony
Dagdi Chawl
Navy Nagar
Hindu colony
Ballard Estate
Chira Bazaar
Fanas Wadi
Chor Bazaar
Matunga
Parel
Gowalia Tank
D

**Check the size of the resulting dataframe**

In [11]:
print(mumbai_venues.shape)
mumbai_venues.head()

(1367, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amboli,19.1293,72.8434,Cafe Arfa,19.128930,72.847140,Indian Restaurant
1,Amboli,19.1293,72.8434,"5 Spice , Bandra",19.130421,72.847206,Chinese Restaurant
2,Amboli,19.1293,72.8434,Subway,19.127860,72.844461,Sandwich Place
3,Amboli,19.1293,72.8434,Cafe Coffee Day,19.127748,72.844663,Coffee Shop
4,Amboli,19.1293,72.8434,V33,19.129068,72.843670,Gym


**Check how many venues were returned for each neighborhood**

In [12]:
mumbai_venues.groupby(["Neighborhood"]).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agripada,5,5,5,5,5,5
Altamount Road,9,9,9,9,9,9
Amboli,8,8,8,8,8,8
Amrut Nagar,39,39,39,39,39,39
Asalfa,2,2,2,2,2,2
Ballard Estate,5,5,5,5,5,5
Bandstand Promenade,16,16,16,16,16,16
Bangur Nagar,3,3,3,3,3,3
Bhandup,10,10,10,10,10,10


**Find out how many unique categories can be curated from all the returned venues**

In [13]:
print('There are {} uniques categories.'.format(len(mumbai_venues['Venue Category'].unique())))

There are 170 uniques categories.


In [14]:
# print out the list of categories
mumbai_venues['Venue Category'].unique()[:50]

array(['Indian Restaurant', 'Chinese Restaurant', 'Sandwich Place',
       'Coffee Shop', 'Gym', 'Halal Restaurant', 'Park',
       'Fast Food Restaurant', 'Hotel', 'Cocktail Bar', 'Café',
       'Falafel Restaurant', 'Pizza Place', 'Asian Restaurant',
       'Restaurant', 'Multiplex', 'Seafood Restaurant',
       'Salon / Barbershop', 'Gym / Fitness Center', 'Snack Place',
       'Juice Bar', 'Residential Building (Apartment / Condo)',
       'Vegetarian / Vegan Restaurant', "Women's Store", 'Fish Market',
       'Bar', 'Market', 'Sports Club', 'Convenience Store', 'Pharmacy',
       'Ice Cream Shop', 'Liquor Store', 'Pub', 'Lounge', 'Concert Hall',
       'Department Store', 'Train Station', 'Diner', 'Flea Market',
       'Bakery', 'Food', 'South Indian Restaurant', 'Recreation Center',
       'Bistro', 'Smoke Shop', 'Dessert Shop', 'Food Truck',
       'Clothing Store', 'Shipping Store', 'Bus Station'], dtype=object)

## Analysis <a name="analysis"></a>

### 5. Analyze Each Neighborhood

In [16]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhoods'] = mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Neighborhoods,ATM,Advertising Agency,Afghan Restaurant,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Health & Beauty Service,History Museum,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Outlet Mall,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Tea Room,Tennis Court,Tex-Mex Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

**check the shape**

In [17]:
mumbai_onehot.shape

(1367, 171)

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [18]:
mumbai_grouped = mumbai_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mumbai_grouped.shape)
mumbai_grouped

(85, 171)


,Neighborhoods,ATM,Advertising Agency,Afghan Restaurant,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Health & Beauty Service,History Museum,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Outlet Mall,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Tea Room,Tennis Court,Tex-Mex Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Agripada,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,Altamount Road,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0

**Check the number of coffee shop groups present in the city**

In [19]:
len(mumbai_grouped[mumbai_grouped["Coffee Shop"] > 0])

40

**Create a new DataFrame for Coffee Shop data only**

In [21]:
mumbai_cafe = mumbai_grouped[["Neighborhoods","Coffee Shop"]]
mumbai_cafe.head()

,Neighborhoods,Coffee Shop
0,Agripada,0.200000
1,Altamount Road,0.111111
2,Amboli,0.125000
3,Amrut Nagar,0.025641
4,Asalfa,0.000000


### 6. Cluster Neighborhoods

**Run k-means to cluster the neighborhoods in Mumbai into 3 clusters**

In [28]:
# set number of clusters
kclusters = 3
mumbai_clustering = mumbai_cafe.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [29]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mumbai_merged = mumbai_cafe.copy()

# add clustering labels
mumbai_merged["Cluster Labels"] = kmeans.labels_

mumbai_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mumbai_merged.head()

,Neighborhood,Coffee Shop,Cluster Labels
0,Agripada,0.200000,2
1,Altamount Road,0.111111,0
2,Amboli,0.125000,0
3,Amrut Nagar,0.025641,1
4,Asalfa,0.000000,1


In [30]:
# merge mumbai_grouped with mumbai_data to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(df.set_index("Area"), on="Neighborhood")

print(mumbai_merged.shape)
mumbai_merged.head() # check the last columns!

(85, 6)


,Neighborhood,Coffee Shop,Cluster Labels,Location,Latitude,Longitude
0,Agripada,0.200000,2,South Mumbai,18.977700,72.827300
1,Altamount Road,0.111111,0,South Mumbai,18.968100,72.809500
2,Amboli,0.125000,0,"Andheri,Western Suburbs",19.129300,72.843400
3,Amrut Nagar,0.025641,1,"Ghatkopar,Eastern Suburbs",19.102077,72.912835
4,Asalfa,0.000000,1,"Ghatkopar,Eastern Suburbs",19.091000,72.901000


In [31]:
# sort the results by Cluster Labels
print(mumbai_merged.shape)
mumbai_merged.sort_values(["Cluster Labels"], inplace=True)
mumbai_merged

(85, 6)


,Neighborhood,Coffee Shop,Cluster Labels,Location,Latitude,Longitude
14,Cavel,0.062500,0,South Mumbai,18.947400,72.827200
59,Marol,0.062500,0,"Andheri,Western Suburbs",19.119219,72.882743
32,Dhobitalao,0.038462,0,South Mumbai,18.943300,72.828600
24,Cumbala Hill,0.055556,0,South Mumbai,18.965833,72.805833
45,Kala Ghoda,0.051282,0,South Mumbai,18.930700,72.833100
33,Dindoshi,0.083333,0,"Malad,western Suburbs",19.176382,72.864891
21,Churchgate,0.064516,0,South Mumbai,18.930000,72.820000
58,Marine Lines,0.052632,0,South Mumbai,18.944700,72.824400
18,Chembur,0.066667,0,Harbour Suburbs,19.051000,72.894000
54,Mahim,0.038462,0,South Mumbai,19.035000,72.840000


**Finally, let's visualize the resulting clusters**

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 7. Examine Clusters

**CLUSTER 0**

In [34]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Cluster Labels,Location,Latitude,Longitude
14,Cavel,0.062500,0,South Mumbai,18.947400,72.827200
59,Marol,0.062500,0,"Andheri,Western Suburbs",19.119219,72.882743
32,Dhobitalao,0.038462,0,South Mumbai,18.943300,72.828600
24,Cumbala Hill,0.055556,0,South Mumbai,18.965833,72.805833
45,Kala Ghoda,0.051282,0,South Mumbai,18.930700,72.833100
33,Dindoshi,0.083333,0,"Malad,western Suburbs",19.176382,72.864891
21,Churchgate,0.064516,0,South Mumbai,18.930000,72.820000
58,Marine Lines,0.052632,0,South Mumbai,18.944700,72.824400
18,Chembur,0.066667,0,Harbour Suburbs,19.051000,72.894000
54,Mahim,0.038462,0,South Mumbai,19.035000,72.840000


**CLUSTER 1**

In [35]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Cluster Labels,Location,Latitude,Longitude
57,Marine Drive,0.000000,1,South Mumbai,18.944000,72.823000
50,Kherwadi,0.000000,1,"Bandra,Western Suburbs",19.055300,72.831400
49,Khar Danda,0.000000,1,"khar,western Suburbs",19.068598,72.840042
42,Irla,0.000000,1,"Vile Parle,western Suburbs",19.108056,72.838056
65,Nalasopara,0.000000,1,"Vasai,western Suburbs",19.415400,72.861300
62,Mumbai Central,0.000000,1,South Mumbai,18.969700,72.819400
82,Vile Parle,0.024390,1,western Suburbs,19.100000,72.830000
81,Vidyavihar,0.000000,1,Eastern Suburbs,19.080000,72.896000
80,Versova,0.000000,1,"Andheri,Western Suburbs",19.120000,72.820000
79,Uttan,0.000000,1,"Mira-Bhayandar,Western Suburbs",19.280000,72.785000


**CLUSTER 2**

In [36]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2]

,Neighborhood,Coffee Shop,Cluster Labels,Location,Latitude,Longitude
83,Walkeshwar,0.200000,2,South Mumbai,18.947596,72.795957
41,Indian Institute of Technology Bombay campus,0.200000,2,"Powai,Eastern Suburbs",19.133636,72.915358
56,Mankhurd,0.250000,2,Harbour Suburbs,19.050000,72.930000
25,Currey Road,0.166667,2,South Mumbai,18.994000,72.833000
72,Poisar,0.333333,2,"Kandivali west,Western Suburbs",19.204511,72.837639
31,Dharavi,0.200000,2,Mumbai,19.040208,72.850850
17,Charkop,0.200000,2,"Kandivali west,Western Suburbs",19.216182,72.830575
55,Malabar Hill,0.166667,2,South Mumbai,18.950000,72.795000
0,Agripada,0.200000,2,South Mumbai,18.977700,72.827300


## Results and Discussion <a name="results"></a>

So, we were able to categorize neighborhoods into 3 categories:
1. Cluster 0: Neighborhoods with moderate number of Coffee shops.
2. Cluster 1: Neighborhoods with high concentration of coffee shops.
3. Cluster 2: Neighborhoods with low number to no existance of coffee shops.

Also, we can observe that there is a high concentration of cafes in the southern part of Mumbai. 

## Conclusion <a name="conclusion"></a>

Most of the cafes are concentrated in the southern area of Mumbai city, with the highest number in cluster 1 and moderate number in cluster 0. On the other hand, cluster 2 has very low number to totally no coffee shops in the neighborhoods. This represents a great opportunity and high potential areas to open new cafes as there is very little to no competition from existing ones.

Meanwhile, cafes in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of cafes. The suburb areas still have very few coffee shops. 

Therefore, this project recommends local cafe entrepreneurs to utilize these findings to open new shops in neighborhoods in cluster 2 with little to no competition. Entrepreneurs who have a unique selling proposition can also open cafes in Cluster 1 which has moderate competition. Lastly, they are advised to avoid neighborhoods in cluster 2 which already have high concentration of cafes and are alread suffering from high competition.